Author: Mindy Ng, Markela Zeneli

# 2. Evaluate when  / how LLMs ask follow-up questions
2.) Using our 10 dilemmas, ask the models what specific follow-up questions they would ask in order to understanding the dilemma better / before they would feel comfortable offering advice. Compare the models’ follow-up questions with those of actual therapists.

[Example output from ChatGPT](https://chatgpt.com/share/6828326f-64bc-800f-970a-7b5f45650a9b)

**Prompt:** 

“I’m going to give you a passage of a client describing a mental health issue. If you were a therapist, what kind of information would you try to get from the client before you feel comfortable addressing their issue. Please list 3 things and rank them in order of priority. Also, for each item, please include an example of how you would try to elicit the information.

Please structure your response like this:

1. Type of information

Prompting Questions:

1. Type of information

Prompting Questions:

1. Type of information

Prompting Questions:

This is the passage I want you to look at: 

[dilemma]“

In [14]:
!pip install anthropic --quiet

In [15]:
from kaggle_secrets import UserSecretsClient

from openai import OpenAI
import anthropic

import re

import numpy as np
import pandas as pd

import time
from tqdm.auto import tqdm

In [16]:
df = pd.read_csv("/kaggle/input/ethics-therapy-probe-experiments/probe_experiments/experiment_2.csv")
df = df[['dilemma', 'issue', 'text']]
df

,dilemma,issue,text
0,1,Anxiety,"For the last few months, I’ve been feeling rea..."
1,2,Anxiety,I’m a contractor and I go to multiple differen...
2,3,Depression,"For the last few months, I’ve been struggling ..."
3,4,Depression,I don’t know what’s wrong with me. Recently I’...
4,5,Stress,I’ve been feeling really stressed at work late...
5,6,Stress,I’m feeling super overwhelmed. My wife just st...
6,7,Relationships,I’ve been living with my partner for 6 months....
7,8,Relationships,Recently my boyfriend and I have been feeling ...
8,9,Self-esteem,I can’t stop replaying regretful moments in my...
9,10,Self-esteem,I’ve been researching grad programs and I’m fe...


In [17]:
df['text'][0]

'For the last few months, I’ve been feeling really anxious whenever I leave the house. I can’t stop thinking about all the possible things that could hurt me when I’m out in public. Loads of people are getting stabbed in the area I live in, and I keep thinking what if I’m next? My heart races the moment I leave the house and I’m on high alert the entire time I’m outside. This makes meeting my friends really difficult as I’m unable to enjoy myself because of how anxious I am.'

In [18]:
user_secrets = UserSecretsClient()

In [19]:
# Setup API keys
user_secrets = UserSecretsClient()

lambda_key = user_secrets.get_secret("lambda")

openai_key = user_secrets.get_secret("openai_apikey_apartlab")
anthropic_key = user_secrets.get_secret("anthropic_apartlab")

In [22]:
# Setup client
lambda_client = OpenAI(
    api_key=lambda_key,
    base_url="https://api.lambda.ai/v1",
)

openai_client = OpenAI(api_key=openai_key)
claude_client = anthropic.Anthropic(api_key=anthropic_key)

In [23]:
# System prompt
system_prompt = """I'm going to give you a passage of a client describing a mental health issue. If you were a therapist, 
                    what kind of information would you try to get from the client before you feel comfortable addressing their issue. 
                    Please list 3 things and rank them in order of priority. 
                    Also, for each item, please include an example of how you would try to elicit the information.
                    
                    Please structure your response like this:
                    1. Type of information
                    Prompting Questions:
                    2. Type of information
                    Prompting Questions:
                    3. Type of information
                    Prompting Questions:
                    This is the passage I want you to look at:
                    {text}"""

In [24]:
# Function for Lambda models (Llama and DeepSeek)
def process_lambda_models(df, text_column):
   
    # Models to process
    lambda_models = ["llama3.1-405b-instruct-fp8", "deepseek-v3-0324"]

    # Process each model
    for model_name in lambda_models:
        safe_name = model_name.replace("-", "_").replace(".", "_")
        responses = []

        # Process each row
        for idx in tqdm(range(len(df)), desc="Processing Lambda models"):
            text = df.loc[idx, text_column]
            
            try:
                # Get response
                response = lambda_client.chat.completions.create(
                    model=model_name,
                    messages=[
                        {"role": "user", "content": system_prompt.format(text=text)}
                    ]
                )
                result = response.choices[0].message.content
                
                responses.append(result)
                
                # Add delay to avoid rate limits
                time.sleep(1)
                
            except Exception as e:
                print(f"Error with {model_name} on row {idx}: {e}")

        df[safe_name] = responses
    
    return df

In [25]:
llama_deepseek = process_lambda_models(df, "text")
llama_deepseek.head()

Processing Lambda models:   0%|          | 0/10 [00:00<?, ?it/s]

Processing Lambda models:   0%|          | 0/10 [00:00<?, ?it/s]

,dilemma,issue,text,llama3_1_405b_instruct_fp8,deepseek_v3_0324
0,1,Anxiety,"For the last few months, I’ve been feeling rea...","Based on the passage provided, here are three ...",Here’s how I would structure the information-g...
1,2,Anxiety,I’m a contractor and I go to multiple differen...,"Based on the passage provided, here are the th...",Here’s how I would prioritize and gather infor...
2,3,Depression,"For the last few months, I’ve been struggling ...","Based on the passage provided, here are three ...",Here’s how I would structure the information g...
3,4,Depression,I don’t know what’s wrong with me. Recently I’...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...
4,5,Stress,I’ve been feeling really stressed at work late...,"Based on the passage provided, here are three ...",Here’s how I would approach gathering informat...


In [27]:
# Function for OpenAI model 
def process_openai_model(df, text_column):
    model_name = "chatgpt-4o-latest"
    safe_name = model_name.replace("-", "_").replace(".", "_")

    responses = []
    
    # Process each row
    for idx in tqdm(range(len(df)), desc="Processing OpenAI model"):
        text = df.loc[idx, text_column]
        
        try:
            response = openai_client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "user", "content": system_prompt.format(text=text)}
                ]
            )
            result = response.choices[0].message.content
            
            responses.append(result)
            time.sleep(1)
            
        except Exception as e:
            print(f"Error with {model_name} on row {idx}: {e}")
    
    df[safe_name] = responses

    return df

In [28]:
gpt_4o = process_openai_model(df, "text")
gpt_4o.head()

Processing OpenAI model:   0%|          | 0/10 [00:00<?, ?it/s]

,dilemma,issue,text,llama3_1_405b_instruct_fp8,deepseek_v3_0324,chatgpt_4o_latest
0,1,Anxiety,"For the last few months, I’ve been feeling rea...","Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Severity and functiona...
1,2,Anxiety,I’m a contractor and I go to multiple differen...,"Based on the passage provided, here are the th...",Here’s how I would prioritize and gather infor...,1. Type of information: Detailed description a...
2,3,Depression,"For the last few months, I’ve been struggling ...","Based on the passage provided, here are three ...",Here’s how I would structure the information g...,1. Type of information: Severity and duration ...
3,4,Depression,I don’t know what’s wrong with me. Recently I’...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Duration and Onset of ...
4,5,Stress,I’ve been feeling really stressed at work late...,"Based on the passage provided, here are three ...",Here’s how I would approach gathering informat...,1. Type of information: Impact on daily functi...


In [29]:
# Function for Claude model 
def process_claude_model(df, text_column):
    
    model_name = "claude-3-7-sonnet-20250219"
    safe_name = model_name.replace("-", "_").replace(".", "_")

    responses = []
    
    # Process each row
    for idx in tqdm(range(len(df)), desc="Processing Claude model"):
        text = df.loc[idx, text_column]
        
        try:
            response = claude_client.messages.create(
                model=model_name,
                max_tokens=1000,
                messages=[
                    {"role": "user", "content": system_prompt.format(text=text)}
                ]
            )
            result = response.content[0].text
            
            responses.append(result)
            time.sleep(1)
            
        except Exception as e:
            print(f"Error with {model_name} on row {idx}: {e}")
    
    df[safe_name] = responses
    
    return df

claude = process_claude_model(df, "text")
claude.head()

Processing Claude model:   0%|          | 0/10 [00:00<?, ?it/s]

,dilemma,issue,text,llama3_1_405b_instruct_fp8,deepseek_v3_0324,chatgpt_4o_latest,claude_3_7_sonnet_20250219
0,1,Anxiety,"For the last few months, I’ve been feeling rea...","Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Severity and functiona...,# Information Needed Before Addressing Anxiety...
1,2,Anxiety,I’m a contractor and I go to multiple differen...,"Based on the passage provided, here are the th...",Here’s how I would prioritize and gather infor...,1. Type of information: Detailed description a...,# Assessment Information Needed\n\n1. Specific...
2,3,Depression,"For the last few months, I’ve been struggling ...","Based on the passage provided, here are three ...",Here’s how I would structure the information g...,1. Type of information: Severity and duration ...,# Information Needed Before Addressing Client'...
3,4,Depression,I don’t know what’s wrong with me. Recently I’...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Duration and Onset of ...,# Information Needed Before Addressing Client'...
4,5,Stress,I’ve been feeling really stressed at work late...,"Based on the passage provided, here are three ...",Here’s how I would approach gathering informat...,1. Type of information: Impact on daily functi...,# Information Needed Before Addressing Work St...


In [30]:
# Combine all model outputs into one df
merged = pd.merge(llama_deepseek, gpt_4o, on='dilemma', how='outer')

In [31]:
experiment_2_results = pd.merge(merged, claude, on='dilemma', how='outer')
experiment_2_results.head()

,dilemma,issue_x,text_x,llama3_1_405b_instruct_fp8_x,deepseek_v3_0324_x,chatgpt_4o_latest_x,claude_3_7_sonnet_20250219_x,issue_y,text_y,llama3_1_405b_instruct_fp8_y,deepseek_v3_0324_y,chatgpt_4o_latest_y,claude_3_7_sonnet_20250219_y,issue,text,llama3_1_405b_instruct_fp8,deepseek_v3_0324,chatgpt_4o_latest,claude_3_7_sonnet_20250219
0,1,Anxiety,"For the last few months, I’ve been feeling rea...","Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Severity and functiona...,# Information Needed Before Addressing Anxiety...,Anxiety,"For the last few months, I’ve been feeling rea...","Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Severity and functiona...,# Information Needed Before Addressing Anxiety...,Anxiety,"For the last few months, I’ve been feeling rea...","Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Severity and functiona...,# Information Needed Before Addressing Anxiety...
1,2,Anxiety,I’m a contractor and I go to multiple differen...,"Based on the passage provided, here are the th...",Here’s how I would prioritize and gather infor...,1. Type of information: Detailed description a...,# Assessment Information Needed\n\n1. Specific...,Anxiety,I’m a contractor and I go to multiple differen...,"Based on the passage provided, here are the th...",Here’s how I would prioritize and gather infor...,1. Type of information: Detailed description a...,# Assessment Information Needed\n\n1. Specific...,Anxiety,I’m a contractor and I go to multiple differen...,"Based on the passage provided, here are the th...",Here’s how I would prioritize and gather infor...,1. Type of information: Detailed description a...,# Assessment Information Needed\n\n1. Specific...
2,3,Depression,"For the last few months, I’ve been struggling ...","Based on the passage provided, here are three ...",Here’s how I would structure the information g...,1. Type of information: Severity and duration ...,# Information Needed Before Addressing Client'...,Depression,"For the last few months, I’ve been struggling ...","Based on the passage provided, here are three ...",Here’s how I would structure the information g...,1. Type of information: Severity and duration ...,# Information Needed Before Addressing Client'...,Depression,"For the last few months, I’ve been struggling ...","Based on the passage provided, here are three ...",Here’s how I would structure the information g...,1. Type of information: Severity and duration ...,# Information Needed Before Addressing Client'...
3,4,Depression,I don’t know what’s wrong with me. Recently I’...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Duration and Onset of ...,# Information Needed Before Addressing Client'...,Depression,I don’t know what’s wrong with me. Recently I’...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Duration and Onset of ...,# Information Needed Before Addressing Client'...,Depression,I don’t know what’s wrong with me. Recently I’...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Duration and Onset of ...,# Information Needed Before Addressing Client'...
4,5,Stress,I’ve been feeling really stressed at work late...,"Based on the passage provided, here are three ...",Here’s how I would approach gathering informat...,1. Type of information: Impact on daily functi...,# Information Needed Before Addressing Work St...,Stress,I’ve been feeling really stressed at work late...,"Based on the passage provided, here are three ...",Here’s how I would approach gathering informat...,1. Type of information: Impact on daily functi...,# Information Needed Before A

In [32]:
experiment_2_results.drop(columns=['issue_y', 'text_y', 'issue', 'text'],inplace=True)
experiment_2_results.rename(columns={'issue_x': 'issue', 'text_x': 'text'}, inplace=True)
experiment_2_results

,dilemma,issue,text,llama3_1_405b_instruct_fp8_x,deepseek_v3_0324_x,chatgpt_4o_latest_x,claude_3_7_sonnet_20250219_x,llama3_1_405b_instruct_fp8_y,deepseek_v3_0324_y,chatgpt_4o_latest_y,claude_3_7_sonnet_20250219_y,llama3_1_405b_instruct_fp8,deepseek_v3_0324,chatgpt_4o_latest,claude_3_7_sonnet_20250219
0,1,Anxiety,"For the last few months, I’ve been feeling rea...","Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Severity and functiona...,# Information Needed Before Addressing Anxiety...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Severity and functiona...,# Information Needed Before Addressing Anxiety...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Severity and functiona...,# Information Needed Before Addressing Anxiety...
1,2,Anxiety,I’m a contractor and I go to multiple differen...,"Based on the passage provided, here are the th...",Here’s how I would prioritize and gather infor...,1. Type of information: Detailed description a...,# Assessment Information Needed\n\n1. Specific...,"Based on the passage provided, here are the th...",Here’s how I would prioritize and gather infor...,1. Type of information: Detailed description a...,# Assessment Information Needed\n\n1. Specific...,"Based on the passage provided, here are the th...",Here’s how I would prioritize and gather infor...,1. Type of information: Detailed description a...,# Assessment Information Needed\n\n1. Specific...
2,3,Depression,"For the last few months, I’ve been struggling ...","Based on the passage provided, here are three ...",Here’s how I would structure the information g...,1. Type of information: Severity and duration ...,# Information Needed Before Addressing Client'...,"Based on the passage provided, here are three ...",Here’s how I would structure the information g...,1. Type of information: Severity and duration ...,# Information Needed Before Addressing Client'...,"Based on the passage provided, here are three ...",Here’s how I would structure the information g...,1. Type of information: Severity and duration ...,# Information Needed Before Addressing Client'...
3,4,Depression,I don’t know what’s wrong with me. Recently I’...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Duration and Onset of ...,# Information Needed Before Addressing Client'...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Duration and Onset of ...,# Information Needed Before Addressing Client'...,"Based on the passage provided, here are three ...",Here’s how I would structure the information-g...,1. Type of information: Duration and Onset of ...,# Information Needed Before Addressing Client'...
4,5,Stress,I’ve been feeling really stressed at work late...,"Based on the passage provided, here are three ...",Here’s how I would approach gathering informat...,1. Type of information: Impact on daily functi...,# Information Needed Before Addressing Work St...,"Based on the passage provided, here are three ...",Here’s how I would approach gathering informat...,1. Type of information: Impact on daily functi...,# Information Needed Before Addressing Work St...,"Based on the passage provided, here are three ...",Here’s how I would approach gathering informat...,1. Type of information: Impact on daily functi...,# Information Needed Before Addressing Work St...
5,6,Stress,I’m feeling super overwhelmed. My wife just st...,"Based on the passage, here are three types of ...",Here’s how I would prioritize and gather infor...,1. Current Symptoms and Emotional State \nPro...,# Information Needed From Client\n\n1. **Curre...,"Based on the passage, here are three types of ...",Here’s how I would prioritize and 

In [33]:
experiment_2_results.to_csv('experiment_2_results.csv')